In [1]:
%run ../00_default_options.ipynb

In [2]:
from robotehr.api.training import get_training_results
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

[2020-06-20 11:20:26,516] INFO in __init__: RobotEHR API


In [3]:
cohort = Cohort.load(id=1)

In [4]:
onset_dataframe = OnsetDataFrame.load(id=17)

In [5]:
feature_pipeline = FeaturePipeline.load(id=21)

In [6]:
import morpher.config
from morpher.jobs import *

Using TensorFlow backend.


In [7]:
regexes = [
    'any__Diagnosis__',
    'any__Procedure__', 
    'any__Drug__', 
    'any__Material__', 
    'any__Encounter__', 
    'any__AlcoholUse__', 
    'any__DrugUse__', 
    'any__TobaccoUse__',
    'min__height__', 
    'min__weight__', 
    'min__vitalsign__', 
    'min__measurement__', 
    'min__labvalue__', 
    'max__height__', 
    'max__weight__', 
    'max__vitalsign__', 
    'max__measurement__', 
    'max__labvalue__'
]

agg_func_regex = ""
for r in regexes:
    agg_func_regex += r + "|"
agg_func_regex = agg_func_regex[:-1]

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]] = label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]].fillna(False)
    
    scaled_data, _ = Scale().execute(data=label_encoded_data, target=target)
    
    imputed_data, _ = Impute().execute(data=scaled_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [8]:
observation_window_ends = [0, 1, 3, 7, 11, 14, 18, 21, 25, 29, 37, 45, 50, 55, 59, 70, 80, 90]

In [9]:
from itertools import product

In [10]:
pipeline_ids = []

In [11]:
for ow_end in observation_window_ends:
    thresholds_numeric = [(x / 100) for x in range(5, 15, 5)]
    observation_windows_numeric = [[x, ow_end] for x in range(-361, ow_end, 90)]
    thresholds_occurring = [0.05]
    observation_windows_occurring = [[-361, ow_end]]

    iterator = product(
        thresholds_numeric,
        observation_windows_numeric,
        thresholds_occurring,
        observation_windows_occurring
    )

    configs = [x for x in iterator]
    targets = [f'urinary_tract_infection_onset_from_{ow_end + 1}_days_after_to_365_days_after']
    algorithms = [morpher.config.algorithms.LR]
    samplers = [morpher.config.samplers.NOSAMPLER]
    
    pipeline = training.execute(
        comment=f'NEW baseline-4 uti post-tx - {ow_end + 1} to 365d',
        version=f'14.0.0-post-tx-{ow_end + 1}',
        cohort=cohort,
        onset_dataframe=onset_dataframe,
        feature_pipeline=feature_pipeline,
        data_loader=data_loader,
        observation_iterator=configs,
        targets=targets,
        algorithms=algorithms,
        samplers=samplers,
        feature_type_occurring="occurring",
        feature_type_numeric="numeric_binned",
        bin_size=30,
        rfe__run=True,
        rfe__step_size=50
    )
    pipeline_ids.append({'id': pipeline.id, 'comment': pipeline.comment})

Columns (7,23,34,42,54,55,56,57,133,138,146) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (50,122,124,130,182,189,224,245,327,339,443,490,575,595) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,18,37,46,49,72,89,96,103,118,139) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,19,43,53,56,79,96,103,110,125,141,184,281,282,295,299,324) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f612830c2e8>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


Fitting estimator with 4306 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 44.256ms

Fitting estimator with 4256 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 980.529ms

Fitting estimator with 4206 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 89.163ms

Fitting estimator with 4156 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 26.625ms

Fitting estimator with 4106 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 958.283ms

Fitting estimator with 4056 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 984.84ms

Fitting estimator with 4006 features.
*** Training of model 'LogisticRegression' started.
*** Training of clas

*** Training of classifier ready. Time elapsed: 212.887ms

Fitting estimator with 1506 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 191.007ms

Fitting estimator with 1456 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 171.745ms

Fitting estimator with 1406 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 170.908ms

Fitting estimator with 1356 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 164.574ms

Fitting estimator with 1306 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 159.266ms

Fitting estimator with 1256 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 158.214ms

Fitting estimator with 1206 features.
*** Train

...

*** Training of classifier ready. Time elapsed: 169.551ms

Fitting estimator with 584 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 239.595ms

Fitting estimator with 534 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 276.957ms

Fitting estimator with 484 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 585.636ms

Fitting estimator with 434 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 756.041ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 628.18ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 720.581ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 286.934ms

*** Training of m

Columns (6,15,41,64,73,81,86,102,146,223,266,272,281) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (146,147,171,194,298,299,346,356,378,510,512,533,670,761,804,830,832,846,853,855,860,864,925,942,944,959,964) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,18,37,83,92,119,141,189,197,205,212,226,301) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,19,43,91,100,127,149,197,205,213,284,294,337,387,393,432,440,521) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f6127cfe550>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


Fitting estimator with 2388 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 238.242ms

Fitting estimator with 2338 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 207.797ms

Fitting estimator with 2288 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 686.889ms

Fitting estimator with 2238 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 263.434ms

Fitting estimator with 2188 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 339.829ms

Fitting estimator with 2138 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 798.981ms

Fitting estimator with 2088 features.
*** Training of model 'LogisticRegression' started.
*** Training of 

In [12]:
from pprint import pprint

In [16]:
pprint(pipeline_ids)

[{'comment': 'NEW baseline-4 uti post-tx - 1 to 365d', 'id': 337},
 {'comment': 'NEW baseline-4 uti post-tx - 2 to 365d', 'id': 338},
 {'comment': 'NEW baseline-4 uti post-tx - 4 to 365d', 'id': 339},
 {'comment': 'NEW baseline-4 uti post-tx - 8 to 365d', 'id': 340},
 {'comment': 'NEW baseline-4 uti post-tx - 12 to 365d', 'id': 341},
 {'comment': 'NEW baseline-4 uti post-tx - 15 to 365d', 'id': 342},
 {'comment': 'NEW baseline-4 uti post-tx - 19 to 365d', 'id': 343},
 {'comment': 'NEW baseline-4 uti post-tx - 22 to 365d', 'id': 344},
 {'comment': 'NEW baseline-4 uti post-tx - 26 to 365d', 'id': 345},
 {'comment': 'NEW baseline-4 uti post-tx - 30 to 365d', 'id': 346},
 {'comment': 'NEW baseline-4 uti post-tx - 38 to 365d', 'id': 347},
 {'comment': 'NEW baseline-4 uti post-tx - 46 to 365d', 'id': 348},
 {'comment': 'NEW baseline-4 uti post-tx - 51 to 365d', 'id': 349},
 {'comment': 'NEW baseline-4 uti post-tx - 56 to 365d', 'id': 350},
 {'comment': 'NEW baseline-4 uti post-tx - 60 to 365

In [15]:
[p['id'] for p in pipeline_ids]

[337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354]